# PSPG 245B CDD Workshop - Naive Bayes Classifier

In this notebook, you will be working with data from the ChEMBL dataset of bioactive molecules to train a Naive Bayes classifier to predict protein target given drug structure. This code takes in a file containing various cancer-related compounds (SMILES), along with two reference files containing data on molecules and targets from ChEMBL. We will load the data, convert the SMILES into molecular fingerprint representations of molecules using RDKIT, and train a machine-learning model called a Naive Bayesian Classifier to predict the protein targets of each compound.

#### Import modules - necessary Python packages we will be using.

Confirm that your environment has been installed correctly by running the cell below; either selecting "Run" from the above menu or hitting Shift + Enter.

In [1]:
## RUN THIS CELL ##
# Jupyter Display
from IPython.core.display import display,HTML
display(HTML("<style>.container {width:85% !important;} </style>"))

# Standard Python Tools
import sys, os, operator
import warnings

# File I/O
import csv
import gzip

# Custom functions
from utils import map_target_identifiers, flatten_list, view_target_dist

# Data handling modules
import numpy as np
import pandas as pd

# Sklearn Modules
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

# Chemical Handling Modules
from rdkit import Chem
from rdkit import DataStructs
from rdkit.Chem import AllChem, Draw

# Vizualization Modules
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')

from IPython.display import Image


## Directions
Fix the functions so that our handler function below will work.

**Note_1:** Not all the functions need to be adjusted. Those that work as is, have a comment above mentioning the function does not need to be altered.

**Note_2:** The Jupyter notebook stores all variables created in memory unless explicitely deleted. Thus if you name a variable something and change the name in the same cell, the original variable will STILL be there. This can cause problems if you forget to change all instances of the initial variable later in your script. The easiest way to not worry about this is to restart the kernel, which will flush the memory. However you will have to reload every cell again.

**Note_3:** Ask questions! We are here to help :)

# Part 1: Working with molecules and molecular representations in RDKit

## Molecular representations

Molecules can be represented in a number of different ways, from string representations like SMILES, SMIRKS, and InChi, to fingerprint represetations, to graphs, and lots of others. For the purposes of this workshop, we will be working with SMILES and extended-connectivity fingerprints.

### - SMILES

SMILES (Simplified molecular-input line-entry system) proposed by Weininger (1988) is a popular method to represent molecules as ASCII strings. The string is created by printing out the nodes found on a traversal of the molecular graph. For our purposes it is not particularly important how this traversal happens but note how atoms are represented by their usual element symbol (i.e. carbon by 'C', oxygen by 'O', etc), bonds by the symbols . - = # $ : / \, branching by parentheses, and rings by numbers. RDKit is able to create a Molecule object directly from a SMILES string:

In [ ]:
## RUN THIS CELL ##
paracetemol_str = 'CC(=O)Nc1ccc(O)cc1'
paracetemol_mol = Chem.MolFromSmiles(paracetemol_str)
Draw.MolToImage(paracetemol_mol)

Note that there is a one-to-many mapping between molecules and their SMILES string representations depending on how one traverses the molecular graph! This means that multiple SMILES can all refer to the same molecule.  

When converting a SMILES string to an `RDKit Mol` object (which happened when running `Chem.MolFromSmiles`), RDKit will infer properties of atoms and bonds, which we can then inspect and manipulate. For example, we can iterate through the atoms or bonds:


In [ ]:
## RUN THIS CELL ##
for atm in paracetemol_mol.GetAtoms():
    print(f"Atom element: {atm.GetSymbol()}, atomic number: {atm.GetAtomicNum()}, number of hydrogens {atm.GetTotalNumHs()}")

# Iterate through the bonds..
for bnd in paracetemol_mol.GetBonds():
    print(f"Bond from {bnd.GetBeginAtomIdx()} to {bnd.GetEndAtomIdx()} and is of type {bnd.GetBondType()}.")

###  *Excercise: Convert the following smiles to rdkit mol and find the number of atoms in each*

The skeleton is provided for you, fill in the blanks accordingly.

In [4]:
## RUN THIS CELL ##
molecules = {
    'Glucose': 'OC[C@H]1OC(O)[C@H](O)[C@@H](O)[C@@H]1O',  
    'Thiamine (Vitamin B1)': 'Cc1c(sc[n+]1Cc2cnc([nH]c2=N)C)CCO',   
    'Ibuprofen': 'CC(C)Cc1ccc(cc1)[C@@H](C)C(=O)O'   
}

In [ ]:
## FILL IN THE MISSING ARGUMENTS ##
for mol_name, mol_smi in molecules.items():
    mol = Chem.MolFromSmiles(?)
    num_atoms = len(mol.?) # Hint: what function that we used above gave us a list of atoms?
    print(mol_name, num_atoms)

### - Molecular Fingerprints

Character strings are not the only way to represent molecules. Most simple aachine learning models require a fixed-length numeric input structure. One way of acheiving this for molecules is what's known as a "molecular fingerprint".

The basic idea of molecular fingerprint is as follows:
 1. Assign each atom an initial identifier
 2. Update atom's identifiers based on its neighbors
 3. Remove duplicates
 4. Fold list of identifiers into a n-bit vector (usually 1024)
 
The result of this process is that each bit in the fingerprint represents the presence of a specific moiety or chemical substructure, which can be visualized as:

![fingerprint](https://ars.els-cdn.com/content/image/1-s2.0-S1046202314002631-fx1.jpg)
  
  
You can learn more about molecular fingerprints [here](https://www.sciencedirect.com/science/article/abs/pii/S1046202314002631). In this notebook, we'll be working with Morgan/ECFP fingerprints, as described by [Rogers and Hahn](https://pubs.acs.org/doi/10.1021/ci100050t) and implemented in the RDKit package.


In [ ]:
## RUN THIS CELL ##
mol = Chem.MolFromSmiles(paracetemol_str)
fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius=2, nBits=1024) # find the morgan fingerprint of the molecule at radius and nBits
bstr = fp.ToBitString() # convert to string to print
bstr 

###  *Excercise: For the molecules defined previously, calculate the fingerprint at radius 2, 3. Find the number of on bits for each case using `fp.GetNumOnBits()`*

The skeleton is provided for you, fill in the blanks accordingly.

In [ ]:
## FILL IN THE MISSING ARGUMENTS ##
for radius in [2, 3]:
    for mol_name, mol_smi in molecules.items(): # using same molecules as above
        mol = Chem.MolFromSmiles(mol_smi)
        fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius=?, nBits=1024)
        print(mol_name, ?)

## Molecular Similarity 
### - Tanimoto Coefficients

Along with their use as inputs to ML models, fingerprints can also be used to compute a notion of similarity between different molecules. A common way to to do this is through the Tanimoto similarity between two fingerprints.

$T = \frac{N_{AB}}{N_A + N_B - N_{AB}}$

- $N_A$ represents the number of `on` feature (bits) in A
- $N_B$ represents the number of `on` feature (bits) in B
- $N_{AB}$ represents the number of `on` feature (bits) in both fingerprint A and B

Or in other words, the intersection of `on` bits / union of `on` bits. For identical molecules, these are equivalent, resulting in a TC of 1. Completely disjoint
fingerprints would have a TC of 0.



### *Excercise: For the molecules defined, find the tanimoto similarity using the rdkit TanimotoSimilarity function* 
The skeleton is provided for you, fill in the blanks accordingly.

In [ ]:
## COMPLETE THE FUNCTION ##
def compute_tanimoto_similarity(smiles_1, smiles_2):
    """This function takes in 2 SMILES and input and returns the Tanimoto Similarity between them."""
    #### fill in function ###
    

    return DataStructs.TanimotoSimilarity(?, ?) # Hint - this expects two BitVect, how can we generate them?

In [ ]:
### FILL IN THE MISSING ARGUMENTS ###
mol_names = sorted(list(molecules))
for i, mol_name in enumerate(mol_names):
    for mol2_name in mol_names[i+1:]:
        tc = compute_tanimoto_similarity(molecules[?], molecules[?])
        print(f"Similarity between {mol_name} and {mol2_name}: {tc:.3f}")

### Further reading

**RDKit** learn more about rdkit functions at RDKit [documentation](http://www.rdkit.org/docs/index.html). A tutorial on RDKit that can be useful can be found at [here](http://www.rdkit.org/docs/GettingStartedInPython.html)

Morgan fingerprints are not the only way to compute fingerprints of molecules. RDKit has a series of other fingerprinting methods that can be used. Choosing fingerprints could be an dataset dependent decision, although Rinker and Landrum (2013) find that many popular fingerprints offer fairly similar performance in downstream tasks.

# Part 2: Predicting targets using machine learning

Now that you've gotten a chance to work with RDKit using a few molecular representations, it's time to use these representations to train a model. As discussed above, we will be training our model on the ChEMBL dataset, and then using that trained model to predict the targets of a new set of cancer-related compounds from the NCI-60 dataset. 

### Perform EDA on the NCI-60 dataset
In order to determine which compounds are optimal for treatment, you will need to identify which **breast cancer cell line** is most relevant to your tumor. Before the second portion of the workshop, your goal is to explore the datasets provided by the NCI60 and to make a decision on which cell line(s) to focus on. You will share how you made your decision.

The NCI's genomics and bioinformatics group has created a web portal consolidating all the data and metadata related to the screen that is publicly available and convenient to access. The sites homepage is located here: https://discover.nci.nih.gov/cellminer/home.do We encourage you to examine the upper blue tabs, especially the ones labeled "Data Set Metadata", "Cell Line Metadata", and "Download Data Sets".

In [10]:
# read NC60 screen
nci60 = pd.read_csv("DTP-NCI60_Dataset/dtp_nci60_compounds.csv.gz") # relative path to compounds.csv.gz
nci60 = nci60.melt(id_vars=['PubChem_id', 'SMILES', 'drug_name'], var_name='cell_line', value_name='cpd_activity')

In [ ]:
nci60.head()

In [ ]:
### FILL IN WITH CHOSEN CELL LINE AND NUMBER OF MOLECULES ###
cell_line = ?
n_cpds = ?
# filter
top_cpds = nci60[nci60['cell_line']==cell_line].sort_values(by='cpd_activity', ascending=False)[:n_cpds]
top_cpds = top_cpds[['PubChem_id', 'SMILES', 'drug_name']]
top_cpds.head()
# write to csv
top_cpds.to_csv(f"{cell_line}_top{n_cpds}_cpds.csv", index=False)

### Exercise: Update the following code to plot a histogram of compound activities for your selected cell line.
Whats the range of activities? What's the mean?

In [ ]:
# EDA visualization
cpds_line = nci60[nci60['cell_line'] == cell_line]
cpds_line[?].hist()
np.mean(cpds_line[?])

### Question: What cell line did you choose and why?

Answer here

## Loading and splitting training data

In [13]:
#Python Pandas print options
pd.set_option('display.width', 1000)
pd.set_option('precision', 3)
pd.set_option('max_rows', 15)
pd.set_option('large_repr', 'truncate')
pd.set_option('max_colwidth', 40)
pd.set_option('colheader_justify', 'left')


# Default Directories
BASE_DIR = os.getcwd()

# Default Files
CANCER_CPDS_F = os.path.join(BASE_DIR, 'data', 'cancer_compounds.sample.csv')
CHEMBL_MOLS_F = os.path.join(BASE_DIR, 'data', 'chembl_21_binding_molecules.csv.gz')
CHEMBL_TARGS_F = os.path.join(BASE_DIR, 'data', 'chembl_21_binding_targets.csv.gz')

### Reading our dataset - Utility functions
The following functions perform a number of operations such as reading the dataset, converting the molecules to fingerprints, and getting the drug target for each compound. They are provided for you and do not need to be altered.

In [14]:
## THESE FUNCTION DOES NOT NEED TO BE ALTERED ##
# documentation:
# https://www.rdkit.org/docs/source/rdkit.Chem.rdmolfiles.html#rdkit.Chem.rdmolfiles.MolFromSmiles
# https://rdkit.readthedocs.io/en/latest/GettingStartedInPython.html#morgan-fingerprints-circular-fingerprints
def gen_compound_bitstring(smile, radius=4, nBits=1024):
    """ Generate morgan-fingerprint from compound smile, convert fingerprint to bit-vec"""
    mol = Chem.MolFromSmiles(smile) # convert the smiles to RDKit mol object
    if mol is None:
        return None
    fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius, nBits) # find the morgan fingerprint of the molecule at radius and nBits
    bstr = fp.ToBitString()
    return bstr

def format_training_data_for_naive_bayes(chembl_mol_f, chembl_targ_f):
    """Returns nDim array of bit-vectorized molecules (x-values) and corresponding nDim array of target classes
    that each bit-vector is assigned to (y-values)."""
    bayes_XY_fields = []
    cpid_to_bitVec = {} # cpid = compound chembl_ID
    
    print('Formatting training data for use with Naive Bayesian Classifier...')
    mol_fi, targ_fi = (gzip.open(chembl_mol_f, 'rt'), gzip.open(chembl_targ_f, 'rt'))
    # Iterate through mol_file, transform smile to bit-vector, map cpid to its corresponding bit-vector
    mol_reader = csv.reader(mol_fi)
    next(mol_reader) # circumvent file header
    print('\tTransforming cpd smiles to bit-vectors from file: {}'.format(os.path.basename(chembl_mol_f)))
    for cpid, smi, fp in mol_reader:
        bitVec = np.frombuffer(fp.encode(), 'i1') - 48 # creates numpy array from string of 0's and 1's
        cpid_to_bitVec[cpid] = bitVec
    print('\t\tGenerated {} compound bit-vectors'.format(len(cpid_to_bitVec)))

    # Iterate through targ_file, get all compounds (bitVector) associated with each target, format for bayes learning.
    targ_reader = csv.reader(targ_fi)
    next(targ_reader)
    print("\tMapping target classes to compound bit-vectors from file: {}".format(os.path.basename(chembl_targ_f)))
    for targID, uniprotID, cpd_assocs, targ_desc in targ_reader:
        for cpid in cpd_assocs.split(':'):
            if cpid in cpid_to_bitVec:
                bitVec = cpid_to_bitVec[cpid]
                bayes_XY_fields.append((bitVec, targID))
    
    # Return xvalues (bit-Vectors for each compound), and yvalues (class label of target each compound is associated with)
    xvals, yvals = zip(*bayes_XY_fields)
    print('\t\tGenerated training examples.')
    print('')
    return np.array(xvals), np.array(yvals)

Confirm that the functions above are working correctly by running the following cell, which reads in the ChEMBL molecules and targets and formats them into training data and labels. The training data are fingerprints, and the label for each is the corresponding protein target.

In [16]:
all_data, all_classes = format_training_data_for_naive_bayes(CHEMBL_MOLS_F, CHEMBL_TARGS_F)

Formatting training data for use with Naive Bayesian Classifier...
	Transforming cpd smiles to bit-vectors from file: chembl_21_binding_molecules.csv.gz
		Generated 334291 compound bit-vectors
	Mapping target classes to compound bit-vectors from file: chembl_21_binding_targets.csv.gz
		Generated training examples.



### *Question: How many examples do we have in total?*

### Splitting our dataset

In ML pipelines, it's common to split the data between training and testing sets. This allows us to evaluate the accuracy of our model against known labels before we apply it to new data. To split up our dataset, we will use the built in scikit-learn utility `train_test_split`

In [17]:
training_data, testing_data, training_classes, testing_classes = train_test_split(
    all_data, all_classes, 
    random_state=123, # Seed random seed for reproducibility, always forces the same result
    test_size=0.3)    # 70/30 split

### Training our model: Naive Bayes Classifier

Now that our data are formatted correctly, we can pass them to our machine learning model. In this notebook, we will using a Naive Bayes classifier from Scikit-Learn. A classifier is a machine learning model that is used to discriminate different objects based on input features. 

Naive bayes classifier is based on Bayes theorem:

$P(A|B) = \frac{P(B|A)P(A)}{P(B)}$

for a classification task this can be written as:

$P(y|X) = \frac{P(X|y)P(y)}{P(X)}$

where y is the calss variable and X represents parameters/features. X is not a single feature but could be multiple independent features. 

Based on chain rule:

$P(y|X) = \frac{P(x1|y)P(x2|y)...P(xn|y)P(y)}{P(x1)P(x2)...P(xn)}$

the denominator of the equation does not change and can be removed so that :

$P(y|x_1...,x_n) \propto P(y) \Pi_{i=1}^{n}P(x_i|y) $

if our task is multiclass classification, we need to find the class y with maximum probability:

$ y = argmax_y P(y) \Pi_{i=1}^{n} P(x_i|y) $

we can use maximum a posteri (MAP) estimation to estimate P(y).

**Multinomial Naive Bayes**: implements the naive bayes algorithm for multinomially distributed data, and is one of the two classic naive Bayes variants used in text classification (where data are typically represented as word counts). The distribution is parameterized by vectors $\theta_y = (\theta_{y1}, ... \theta_{yn})$ for each class y where n is the number of features (in text classifcation, the size of vocabulary) and $\theta_{yi}$ is the probability $P(x_i|y)$ of feature $i$ appearing in a sample belonging to class y.

**Bernoulli Naive Bayes**: this classifier is similar to multinomial bayes but the predictions are boolean variables. 

The following function takes in our training input and labels, as well as the option to swap between a Bernoulli or Multinomical NB Classifier. Fill in the missing text to complete the functions. (Hint: We import everything we need at the top of the notebook - what can we use from there?)


In [ ]:
#https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.BernoulliNB.html
def gen_naive_bayes_classifier(training_data, training_classes, Bernoulli=True, downsample=True):
    """Generate an instance of a Naive Bayesian classifier, based on either multivariate Bernoulli distributions or 
    multinomially distributed data. As our training_data consists of binomial input, the Bernoulli Model seems optimal."""
    if downsample:
        warnings.warn('Downsampling data to test model code, will not return valid result.')
        n = len(training_classes)
        sample_idx = np.random.randint(n, size=int(0.01*n))
        training_data = training_data[sample_idx]
        training_classes = training_classes[sample_idx]
    if Bernoulli:
        print('Initiating Bernoulli Naive Bayesian Classifier')
        classifier = ?
        print('\tFitting classifier to training data...')
        classifier.fit(?, ?)
    else:
        print('Initiating Multinomial Naive Bayesian Classifier')
        classifier = ?
        print('\tFitting classifier to training data...')
        classifier.fit(?, ?)
    print('\tClassifier training complete!')
    print('')
    return classifier


Once you've filled in the cell above, train your model by running the following cell. *NOTE*: This can take some time, especially on the full training set! 

In [ ]:
trained_classifier = gen_naive_bayes_classifier(training_data, training_classes, Bernoulli=True, downsample = True) # downsampling to demo for now

### Evaluating model performance

Once we have trained our model, we can check the performance on the test set. In the following cell, we use the trained model to get predictions for our test set, plot the confusion matrix as a heatmap, and compute overall accuracy.

In [ ]:
## FILL IN THE MISSING ARGUMENTS ##
# see documentation (https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.BernoulliNB.html) if unclear
accuracy = trained_classificer.score(?, ?) 
testing_preds = trained_classifier.predict(?)

ax = plt.axes()
sns.heatmap(confusion_matrix(testing_labels, testing_preds), ax = ax) # plotting
ax.set_title(f'Accuracy: {accuracy:.3f}')
plt.show()
plt.savefig("training_set_heatmap.png")

### Finally: Apply to cancer targets

In [1]:
### Utility function, do not chance ###
def predict_top_targets_for_compounds(cpd_f, classifier, targID_to_targName, ntop=10, verbose=True):
    """Load compounds of interest, convert to fp, get top N target predictions for each compound,
    and write to file."""
    base_dir = os.path.dirname(cpd_f)
    base_name = os.path.basename(cpd_f).split('.csv')[0]
    res_ofn = os.path.join(base_dir, base_name+'.predictions.csv')
    header = ['Zinc_ID', 'Targ_ID', 'Targ_Class', 'Probability_Estimate', 'Drug_Name', 'Smile']
    
    # Initiate csv reader and writer
    #with open(cpd_f, 'r') as fi, open(res_ofn, 'w') as fo:
    fi, fo = (open(cpd_f, 'rt'), open(res_ofn, 'w'))
    reader = csv.reader(fi)
    next(reader)
    writer = csv.writer(fo)
    writer.writerow(header)
    # Iterate through cpd_f, generate compound bit-vectors
    for cpid, smile, drugname in reader:
        bstr = gen_compound_bitstring(smile)
        if bstr is None:
            continue
        temp = np.frombuffer(bstr.encode(), 'i1') - 48
        bitVec = temp.reshape(1, -1)
        
        # Generate array of probabilites for bitVec belonging to each target class
        probabilities = classifier.predict_proba(bitVec)
        target_probs = zip(classifier.classes_, probabilities[0])
        
        # Sort by most relevant targets, and write top N targets associated with bitVec to file
        top_targs = sorted(target_probs, key=operator.itemgetter(1), reverse=True)
        if verbose:
            toprint = [(targID_to_targName[targID], prob) for targID,prob in top_targs[0:ntop]]
            print('{}: '.format(cpid), toprint)
            print('')
        for targID, prob in top_targs[0:ntop]:
            writer.writerow([cpid, targID, targID_to_targName[targID], prob, drugname, smile])
    return res_ofn

In [2]:
def predict_targets(cancer_cpds_f,  trained_classifier, ntop=5, verbose=False):
    """Maps targets, formats data, trains a naive bayesian classifier, then predicts and plots top targets, 
    based on cancer-related compounds file provided"""
    chid_to_targName = map_target_identifiers(chembl_targ_f)
    preds_f = predict_top_targets_for_compounds(cancer_cpds_f, trained_classifier, chid_to_targName, ntop=ntop, verbose=verbose)
    view_target_dist(preds_f, figsize=(18,12))
    return preds_f

In [ ]:
preds_f = predict_targets(CANCER_CPDS_F, trained_classifier, ntop=20, Bernoulli=False, verbose=True, test=True)

In [ ]:
Image(filename='cancer_compounds.png') # visualize most common targets